In [46]:
import random
import numpy as np

Often, in big data, we are asked to tell the difference between two samples. Was there really more rain in 2018 than in 2019? Was the temperature hotter in these two years? Did more people ride bikes in Boston than in Seattle? 

There is a mathmatical way in big data where we can answer these questions. 

Imagine a small town.There are two schools. Each school has two classrooms. In the north school, the two classrooms have students who are 12 years old. In the south class room, classroom a has students who are 12 years old, but classroom b has students who are 17 years old. 

Answer the following:
Do you expect a difference in height between the students in the north school (both classrooms has students who are 12 years old)? Why? 

Do you expect a difference in height between the students in the two classrooms in the south school (one class has students who are 12 years old, and one has students who are 17 years old)? Why?

In [35]:
def get_classroom_north_a():
    return [146.1,152.7, 146.3, 142.0, 151.8,
     151.4, 145.1, 153.5, 151.2, 143.5, 158.2,
     150.6, 143.5, 151.6, 149.9, 154.2, 142.4,
     154.6, 154.1, 152.8, 152.4, 155.9, 152.9,
     149.9, 145.0]


In [36]:
def get_classroom_north_b():
    return [142.9, 150.9, 154.0, 146.4, 142.4,
     148.7, 151.4, 154.5, 142.9, 142.5, 152.5,
     151.4, 156.7, 153.9, 148.5, 147.6, 161.9,
     147.6, 145.1, 143.3, 149.5, 149.7, 147.3,
     148.7, 150.4]

In [37]:
def get_classroom_south_a():
    return [141.8, 150.2, 147.6, 146.6, 153.8,
     149.3, 147.6, 158.0, 146.5, 142.7, 142.1,
     146.4, 152.3, 153.3, 154.7, 158.3, 157.6,
     152.3, 155.8, 152.4, 146.4, 153.3, 149.5,
     148.2, 159.3]

In [38]:
def get_classroom_south_b():
    return [169.6, 163.3, 165.4, 177.6, 164.5,
     169.5, 168.9, 168.4, 168.7, 163.3, 163.4,
     165.0, 164.0, 169.9, 173.6, 161.3, 168.5,
     160.8, 162.3, 164.6, 166.3, 163.6, 152.6,
     172.8, 164.4]

The above functions has the heights for 25 students in each classroom. Let's get the mean (average) for each.

In [44]:
print(np.mean(get_classroom_north_a()))
print(np.mean(get_classroom_north_b()))
print(np.mean(get_classroom_south_a()))
print(np.mean(get_classroom_south_b()))


150.064
149.228
150.64
166.092


0.8359999999999843

For the south school, classroom a has a mea of 150.064, and classroom b has a mean of 149.228. That is a difference of .84 cm. 

Answer the following:

Do you still think there is no difference between the two classrooms in the north school? Why? 

The teacher of classroom a claims his students are taller because they ate peanut butter sandwhiches for lunch every day. The students in classroom b ate junk food. Do you think the teacher is correct? Why?

Mabye the difference in heigh between the two classrooms in the north school is just due to randomness. How can we know? 

Imagine an experiment. We take all the students from the north school and we bring them to the cafeteria. There are now 50 students in the cafeteria. We blindfold the teacher. He cannot see, and her radomly chooses 25 students. Now we take the mean height of this group. We also take the mean of the remaining students. 

Answer the following: Do you expect the same result as the mean above (150.064 and 149.228)? Why? Do you expect the mean from either group to be much different from the original (around 150 cm)? Why? 

Now, imagine that we conduct the same experiment in the south classroom. We take all the students who are 12 years old, and all the students who are 17 years old, and bring them into the cafeterial. We mix them. We have a mix of tall and short students. We randomly pick 25 students and take the mean height.

Answer the following:

Do you expect the this mean to be the same as the original for either group (around 150, or 166)? (Remember, we are mixing in short and tall students.) Why? 

In [45]:
def resample_two_samples(sample1, sample2, num_iterations = 100):
    both = sample1 + sample2
    mean1 = []
    mean2 = []
    for i in range(num_iterations):
        random.shuffle(both) 
        new_1 = both[0:len(sample1)] 
        new_2 = both[len(sample1):] 
        mean1.append(np.mean(new_1)) 
        mean2.append(np.mean(new_2)) 
    return mean1, mean2

In [55]:
resampled_means1, resampled_means2 = resample_two_samples(get_classroom_north_a(), get_classroom_north_b())
resampled_means1 = [round(x,1) for x in resampled_means1]
resampled_means2 = [round(x,1) for x in resampled_means2]
print(resampled_means1)
print()
print(resampled_means2)

[150.4, 149.6, 149.3, 149.9, 150.0, 150.3, 150.3, 148.8, 150.4, 150.1, 149.9, 149.2, 149.4, 150.1, 150.2, 149.7, 148.7, 150.0, 149.5, 149.2, 150.0, 150.9, 149.0, 148.4, 149.9, 149.3, 149.9, 150.1, 150.4, 149.2, 149.3, 149.9, 149.7, 151.0, 150.2, 148.9, 149.6, 149.5, 148.9, 149.7, 149.1, 149.0, 149.9, 149.9, 150.0, 150.4, 148.7, 149.6, 149.4, 148.8, 149.4, 150.4, 148.2, 148.3, 150.0, 149.1, 149.6, 149.8, 149.9, 148.8, 149.9, 149.3, 148.7, 149.4, 150.2, 149.0, 149.4, 149.5, 149.9, 149.6, 150.5, 149.1, 150.1, 149.5, 150.1, 149.3, 150.0, 148.4, 148.9, 149.0, 150.8, 150.5, 149.9, 149.0, 149.5, 148.5, 150.4, 149.5, 149.1, 149.9, 149.3, 149.7, 148.9, 148.8, 149.1, 149.4, 149.9, 148.8, 150.3, 151.0]

[148.9, 149.7, 150.0, 149.4, 149.3, 149.0, 149.0, 150.5, 148.9, 149.1, 149.4, 150.1, 149.9, 149.2, 149.1, 149.6, 150.6, 149.3, 149.7, 150.1, 149.3, 148.4, 150.3, 150.9, 149.4, 150.0, 149.4, 149.2, 148.9, 150.1, 150.0, 149.3, 149.6, 148.2, 149.1, 150.4, 149.7, 149.8, 150.4, 149.6, 150.2, 150.3, 149

Have a look at the code above. We used Python to conduct the experiment with bringing the students into the cafeteria, and randomly choosing 25. The random.shuffle code mixed the students. We did this experiment 100 times. Each time we did the experiment, we calculated the mean. 

Each of these samples is called a *resampled* mean because we re sampled (in this case, 100 times).

Answer the following. Do not use a calculator! The mean of classroom a was 150. Does the first resampeld mean look any different from that mean? Why?

The mean for classroom b was 149. Does the mean of the second sample look any different from the the resampled mean? Why? 

In [56]:
resampled_means1, resampled_means2 = resample_two_samples(get_classroom_south_a(), get_classroom_south_b())
resampled_means1 = [round(x,1) for x in resampled_means1]
resampled_means2 = [round(x,1) for x in resampled_means2]
print(resampled_means1)
print()
print(resampled_means2)

[157.5, 157.8, 157.9, 158.7, 159.3, 157.0, 155.7, 158.3, 156.8, 158.3, 156.7, 158.5, 159.9, 161.4, 158.8, 159.7, 156.7, 157.5, 157.0, 160.8, 157.6, 158.5, 158.3, 158.8, 158.2, 156.5, 158.3, 159.3, 156.9, 158.7, 158.8, 158.5, 156.5, 158.1, 159.4, 160.1, 158.9, 160.1, 157.2, 157.7, 159.1, 159.0, 158.5, 159.3, 157.0, 157.9, 157.4, 158.4, 159.8, 159.3, 160.6, 159.1, 157.7, 159.1, 156.0, 156.5, 159.1, 156.6, 159.1, 158.5, 157.7, 158.9, 159.6, 156.8, 155.6, 159.0, 156.2, 161.8, 160.7, 160.2, 160.8, 159.8, 159.1, 156.1, 159.8, 157.4, 159.1, 159.9, 158.6, 159.0, 158.4, 158.3, 159.7, 158.1, 157.5, 160.8, 156.4, 161.8, 157.7, 158.3, 156.9, 159.0, 159.3, 159.5, 159.0, 160.9, 158.3, 157.9, 160.0, 160.0]

[159.2, 159.0, 158.9, 158.1, 157.4, 159.8, 161.1, 158.4, 159.9, 158.4, 160.1, 158.2, 156.8, 155.3, 157.9, 157.0, 160.1, 159.3, 159.7, 155.9, 159.1, 158.3, 158.4, 157.9, 158.5, 160.2, 158.4, 157.5, 159.9, 158.1, 157.9, 158.2, 160.2, 158.6, 157.3, 156.7, 157.8, 156.6, 159.5, 159.0, 157.7, 157.7, 158

We resampled the two classrooms from the south school

Answer the following. Does the mean of classroom a (151) look different from the resample a?

Does the the mean of classroom b (166) look different from the resample b? 

Population vs Samples

In statistics, we talk about a sample and the population. The population is *everthing*. For example, the population for the average rainfall in the Amazon is the average for every single year, from the beginnig of time until the end of the world. A sample is the set of year 2000 to 2010. 

Generally we think of our data sets as samples. We can never have all the data. We only have little bits. We have to guess what these little bits mean.

In the above example with students, we can think of each classroom as a sample. We can think of mixing the students as the population. 

If the students come from the same population, then if we mix them up (resample) we would expect to get the same results. But if they came from different populations, we would expect to get different results. 

For the south school, we can think of the students coming from the same population. That is why the resampled means look the same.

But for the north school, the students came from different populations. 

In [62]:
# redo the experiment
resampled_means1, resampled_means2 = resample_two_samples(get_classroom_north_a(), get_classroom_north_b())
diff1 = []
#get the mean of classroom a (150.1)
mean1 = np.mean(get_classroom_north_a())
# find out how different each resampled mean is from the classroom
for i in resampled_means1:
    difference = mean1 - i
    difference = round(difference,1)
    diff1.append(difference)
print(diff1)


[0.5, -0.8, 0.4, -0.4, -0.3, -0.4, 0.5, 0.2, -0.3, 0.7, 0.8, 1.1, 0.8, 0.9, 0.5, 1.0, 0.1, 0.3, 0.3, 0.2, 0.4, 0.9, 0.3, 1.3, 0.7, 0.1, 0.2, 0.3, 0.3, -0.2, 0.7, -0.3, 0.9, 0.1, 0.4, 1.3, -0.2, -0.3, -0.2, 0.5, 0.8, 0.4, 0.7, 1.4, -0.3, -0.3, 0.8, -1.0, -0.5, 0.5, 0.9, -0.5, 0.2, -0.5, 0.3, 0.5, 1.4, 0.2, 1.8, 1.0, -0.4, 0.8, 0.6, 0.9, 1.5, 1.2, -0.4, 0.6, 1.5, 0.3, 1.2, 0.2, -1.2, 0.1, 0.3, 0.4, -0.4, -0.1, 0.4, 2.3, 0.3, -0.5, 0.3, 0.7, -0.5, 0.1, -0.9, 0.3, 0.6, 0.8, 0.3, 0.8, -0.0, 0.7, 0.7, 0.4, -0.6, 1.2, 1.6, 0.4]


Notice how the difference in the reseampled means is sometimes greater, and sometimes less? 